我们用打了伪标签的数据进行有监督分类，以进行对比

In [1]:
# 如果这个准确率较低，说明了学习属性方法的优越性。

In [2]:
import sys
sys.path.append('/mnt/workspace')

In [3]:
from py_file.M_attri import Att
from py_file.Get_Data import DATA
from py_file.data_set import MyDataSet
from torch.utils.data import DataLoader,Dataset,random_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import numpy as np
import matplotlib.pyplot as plt
import os
import math

In [4]:
# 定义训练的设备
if torch.cuda.is_available():
    device = torch.device('cuda:0') # 只有一张显卡的话，'cuda'和'cuda:0'是一样的
else:
    device = torch.device('cpu')

print(f'使用的设备为：{device}')

使用的设备为：cuda:0


In [5]:
# 这里加载属性是为了方便获取类别的种类
attri = Att()
attri.compute_mul_defect_att()

In [6]:
trans = transforms.Resize(224)  # ResNet模型适合的图片大小为224x244
# 输入的张量需要带着批次维度和通道维度

# 加载数据

In [7]:
train_data_path = '/mnt/workspace/DATA/train_WM.npz'
train_data = np.load(train_data_path)

pseudo_two_data_path = '../data_fake_label/two_fake_label_WM.npz' 
pseudo_two_data = np.load(pseudo_two_data_path)

pseudo_three_data_path = '../data_fake_label/three_fake_label_WM.npz' 
pseudo_three_data = np.load(pseudo_three_data_path)

pseudo_four_data_path = '../data_fake_label/four_fake_label_WM.npz' 
pseudo_four_data = np.load(pseudo_four_data_path)

val_data_path = '/mnt/workspace/DATA/val_WM.npz'
val_data = np.load(val_data_path)

In [8]:
train_label = train_data['label_name']
pseudo_two_label = pseudo_two_data['label_name']
pseudo_three_label = pseudo_three_data['label_name']
pseudo_four_label = pseudo_four_data['label_name']
val_label = val_data['label_name']

In [9]:
train_wm = train_data['denoise_wm']
train_wm_tensor = trans(torch.reshape(torch.tensor(train_wm, dtype=torch.float32),(len(train_wm),1,52,52)))
print(train_wm_tensor.shape, train_label.shape)

pseudo_two_wm = pseudo_two_data['denoise_wm']
pseudo_two_wm_tensor = trans(torch.reshape(torch.tensor(pseudo_two_wm, dtype=torch.float32),(len(pseudo_two_wm),1,52,52)))
print(pseudo_two_wm_tensor.shape, pseudo_two_label.shape)

pseudo_three_wm = pseudo_three_data['denoise_wm']
pseudo_three_wm_tensor = trans(torch.reshape(torch.tensor(pseudo_three_wm, dtype=torch.float32),(len(pseudo_three_wm),1,52,52)))
print(pseudo_three_wm_tensor.shape, pseudo_three_label.shape)

pseudo_four_wm = pseudo_four_data['denoise_wm']
pseudo_four_wm_tensor = trans(torch.reshape(torch.tensor(pseudo_four_wm, dtype=torch.float32),(len(pseudo_four_wm),1,52,52)))
print(pseudo_four_wm_tensor.shape,  pseudo_four_label.shape)

val_wm = val_data['denoise_wm']
val_wm_tensor = trans(torch.reshape(torch.tensor(val_wm, dtype=torch.float32),(len(val_wm),1,52,52)))
print(val_wm_tensor.shape, val_label.shape)

torch.Size([25910, 1, 224, 224]) (25910,)
torch.Size([2593, 1, 224, 224]) (2593,)
torch.Size([4130, 1, 224, 224]) (4130,)
torch.Size([1629, 1, 224, 224]) (1629,)
torch.Size([3700, 1, 224, 224]) (3700,)


In [10]:
# 转换为列表的形式，方便后续拼接
pseudo_two_wm = list(pseudo_two_wm_tensor)
pseudo_two_label = list(pseudo_two_label)
print(len(pseudo_two_wm))
print(pseudo_two_wm[10].shape)

pseudo_three_wm = list(pseudo_three_wm_tensor)
pseudo_three_label = list(pseudo_three_label)
print(len(pseudo_three_wm))
print(pseudo_three_wm[10].shape)

pseudo_four_wm = list(pseudo_four_wm_tensor)
pseudo_four_label = list(pseudo_four_label)
print(len(pseudo_four_wm))
print(pseudo_four_wm[10].shape)

2593
torch.Size([1, 224, 224])
4130
torch.Size([1, 224, 224])
1629
torch.Size([1, 224, 224])


In [11]:
del pseudo_two_wm_tensor, pseudo_three_wm_tensor, pseudo_four_wm_tensor

In [12]:
train_label_oh = train_data['label_one_hot']
# 通过one_hot标签给数据分为单缺陷、双缺陷、三缺陷以及四缺陷

train_single_wm = []  # 先定义列表，然后转换为tensor
train_single_label = []

train_two_wm = []
train_two_label = []

train_three_wm = []
train_three_label = []

train_four_wm = []
train_four_label = []
for i in range(len(train_label_oh)):
    if train_label_oh[i].sum() <= 1:
        train_single_wm.append(np.array(train_wm_tensor[i]))
        train_single_label.append(train_label[i])

    elif train_label_oh[i].sum() == 2:
        train_two_wm.append(np.array(train_wm_tensor[i]))
        train_two_label.append(train_label[i])

    elif train_label_oh[i].sum() == 3:
        train_three_wm.append(np.array(train_wm_tensor[i]))
        train_three_label.append(train_label[i])

    elif train_label_oh[i].sum() == 4:
        train_four_wm.append(np.array(train_wm_tensor[i]))
        train_four_label.append(train_label[i])


del train_data,train_wm_tensor

In [13]:
train_wm = train_single_wm + pseudo_two_wm + pseudo_three_wm + pseudo_four_wm
train_label = train_single_label + pseudo_two_label + pseudo_three_label + pseudo_four_label

train_wm_tensor = torch.tensor(np.array(train_wm), dtype=torch.float32)
train_label = np.array(train_label)

print(train_wm_tensor.shape, train_label.shape)
print(val_wm_tensor.shape, val_label.shape)

torch.Size([13962, 1, 224, 224]) (13962,)
torch.Size([3700, 1, 224, 224]) (3700,)


In [14]:
train_size = len(train_wm_tensor)
val_size = len(val_wm_tensor)

## 将标签的多热编码改为独热编码

In [15]:
label_order = list(attri.total_defect_att.keys())
print(label_order)
print(len(label_order))

['Center', 'Donut', 'Edge_loc', 'Loc', 'Edge_ring', 'Scratch', 'Random', 'Nearfull', 'Normal', 'C+EL', 'C+ER', 'C+L', 'C+S', 'D+EL', 'D+ER', 'D+L', 'D+S', 'EL+L', 'EL+S', 'ER+L', 'ER+S', 'L+S', 'C+EL+L', 'C+EL+S', 'C+ER+L', 'C+ER+S', 'C+L+S', 'D+EL+L', 'D+EL+S', 'D+ER+L', 'D+ER+S', 'D+L+S', 'EL+L+S', 'ER+L+S', 'C+EL+L+S', 'C+ER+L+S', 'D+EL+L+S', 'D+ER+L+S']
38


In [16]:
label_map_dict = {}  # 字符标签映射为one-hot的字典
for i in range(len(label_order)):
    oh = [0 for _ in range(len(label_order))]
    oh[i] = 1
    label_map_dict[label_order[i]] = oh
print(label_map_dict)

{'Center': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'Donut': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'Edge_loc': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'Loc': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'Edge_ring': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'Scratch': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'Random': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'Nearfull': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [17]:
train_label_oh = []
for l in train_label:
    train_label_oh.append(label_map_dict[l])

val_label_oh = []
for l in val_label:
    val_label_oh.append(label_map_dict[l])

In [18]:
train_label_tensor = torch.tensor(train_label_oh, dtype=torch.float32)
val_label_tensor = torch.tensor(val_label_oh, dtype=torch.float32)
print(train_wm_tensor.shape, train_label_tensor.shape)
print(val_wm_tensor.shape, val_label_tensor.shape)

torch.Size([13962, 1, 224, 224]) torch.Size([13962, 38])
torch.Size([3700, 1, 224, 224]) torch.Size([3700, 38])


In [19]:
train_dataset = MyDataSet(train_wm_tensor, train_label_tensor)
val_dataset = MyDataSet(val_wm_tensor, val_label_tensor)

train_size = len(train_dataset)
val_size = len(val_dataset)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# 定义resnet18和训练需要的函数

In [20]:
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

# 替换第一个卷积以适应我们的单通道图像
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(label_order))

print(model)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [21]:
# 预测outputs中预测正确的个数
def predict_num(outputs,labels_oh):
    right = 0
    for i in range(len(outputs)):
        if np.argmax(outputs[i]) == np.argmax(labels_oh[i]):
            right += 1
    
    return right

def get_acc(model, dataloader,data_size):
    model.eval()
    total_right_num = 0  # 记录正确的总个数
    with torch.no_grad():   # 这里要进行验证，不需要修改参数，所以不计算梯度
        for batch_idx, (inputs, labels_oh) in enumerate(dataloader):
            inputs = inputs.to(device)
            labels_oh = labels_oh.to(device)

            outputs = model(inputs)
            outputs = torch.softmax(outputs, dim=1)

            outputs = outputs.cpu().numpy()
            labels_oh = labels_oh.cpu().numpy()
            
            right_num = predict_num(outputs,labels_oh)
            total_right_num += right_num
    acc = total_right_num / data_size
    return acc

In [22]:
# 展示混淆矩阵的函数
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd

def show_cmat(true_label, predict_label, label_order,typ = 'per', size = 10):
    # 生成混淆矩阵
    cm = confusion_matrix(true_label, predict_label, labels=label_order)  # 按照label_order中的顺序给数据排序
    # 将混淆矩阵中的数字转换为百分比
    # 通过将cm中的每个元素除以其所在行的总和（即该类的真实样本数），再乘以100，将其转换为百分比。
    cm_percentage = cm / cm.sum(axis=1, keepdims=True) * 100

    plt.figure(figsize=(size,size))  # 这个要定义在sns.heatmap()前面才可以定义大小
    # 使用seaborn来可视化混淆矩阵
    if typ == 'per':
        # fmt=".1f"参数设置注释的格式为保留一位小数的浮点数, cmap="Blues"指定了颜色映射
        sns.heatmap(pd.DataFrame(cm_percentage, index=label_order, columns=label_order), annot=True, fmt=".1f", cmap="Blues")
        plt.title('Confusion Matrix in Percentage')
    else:
        # fmt="d"表示格式化为整数，
        sns.heatmap(pd.DataFrame(cm, index=label_order, columns=label_order), annot=True, fmt="d", cmap="Blues")
        plt.title('Confusion Matrix in Number')
    
    plt.xlabel('Predicted')
    plt.ylabel('True')
    # 显示图形
    plt.show()

# 获取真实标签
def predict_labels(outputs, labels_oh, label_order):
    pred_labels = []
    rel_labels = []
    for i in range(len(outputs)):
        pred_labels.append(label_order[np.argmax(outputs[i])])
        rel_labels.append(label_order[np.argmax(labels_oh[i])])
    return rel_labels, pred_labels

def get_labels(model, dataloader, label_order):
    label_true = []
    label_pre = []
    with torch.no_grad(): 
        for imgs,labels in dataloader:  
            imgs = imgs.to(device)
            labels = labels.to(device)
            
            outputs = model(imgs)

            outputs = outputs.cpu().numpy()
            labels = labels.cpu().numpy()

            y_true,y_pre = predict_labels(outputs, labels, label_order)
            
            label_true = label_true + y_true
            label_pre = label_pre + y_pre

    return label_true,label_pre

# 开始训练

In [23]:
model = model.to(device)
loss_func = nn.CrossEntropyLoss().to(device=device) 
learning_rate = 1e-2  # 0.01
optimizer = torch.optim.SGD(params=model.parameters(), lr=learning_rate)
epochs = 30

In [24]:
import time
best_acc = 0
No = 0
for i in range(epochs):
    # total_train_steps = 0
    print(f'————第{i+1}轮训练开始————')

    model.train()   # 开始训练
    total_train_loss = 0
    start_time = time.time()
    for imgs,labels in train_loader:   # 这里的label是一个38维的向量

        imgs = imgs.to(device)
        labels = labels.to(device)

        outputs = model(imgs)
        # print(outputs)
        loss = loss_func(outputs, labels)
        total_train_loss = total_train_loss + loss.item()

        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    end_time = time.time()
    print(f'训练时间为：{end_time-start_time}, 总Loss:{total_train_loss}')  # loss是一个tensor数据类型，loss.item()是一个浮点数数据类型
    print(f'****第{i+1}轮训练结束****')


    # 验证步骤开始

    model.eval()   # 开始验证
    total_val_loss = 0
    # with的作用是可以确保代码块执行完毕后，资源被正确释放，也就是使用with，在执行完外码块之后，它会自动地关闭所打开的内容
    # 例如关闭文件、释放线程锁等
    with torch.no_grad():   # 这里要进行验证，不需要修改参数，所以不计算梯度
        for imgs,labels in val_loader: 

            imgs = imgs.to(device)
            labels = labels.to(device)
            
            outputs = model(imgs)
            # 计算损失
            loss = loss_func(outputs,labels)
            total_val_loss = total_val_loss+loss.item()  # loss是一个tensor数据类型，loss.item()是一个浮点数数据类型
        # 计算准确率
        acc = get_acc(model,val_loader,val_size)
        print(f'第{i+1}轮训练后,整体验证集上的Loss:{total_val_loss}')
        print(f'第{i+1}轮训练后,整体验证集上的Accuracy:{acc}')
        if acc > best_acc:  
            best_acc = acc
            No = i+1
            torch.save(obj=model,f='pseudo_supervised.pth')
print(f'训练结束，第{No}轮的模型在验证集上准确率最高，为{best_acc}')


————第1轮训练开始————
训练时间为：14.683267831802368, 总Loss:335.48893225193024
****第1轮训练结束****
第1轮训练后,整体验证集上的Loss:275.08902257680893
第1轮训练后,整体验证集上的Accuracy:0.30405405405405406
————第2轮训练开始————
训练时间为：12.44685435295105, 总Loss:145.0502940416336
****第2轮训练结束****
第2轮训练后,整体验证集上的Loss:213.18583150207996
第2轮训练后,整体验证集上的Accuracy:0.44351351351351354
————第3轮训练开始————
训练时间为：12.357510805130005, 总Loss:102.247353464365
****第3轮训练结束****
第3轮训练后,整体验证集上的Loss:170.03096205554903
第3轮训练后,整体验证集上的Accuracy:0.5827027027027027
————第4轮训练开始————
训练时间为：12.403621435165405, 总Loss:83.73572927713394
****第4轮训练结束****
第4轮训练后,整体验证集上的Loss:176.9027147218585
第4轮训练后,整体验证集上的Accuracy:0.5127027027027027
————第5轮训练开始————
训练时间为：12.435837268829346, 总Loss:72.44981379806995
****第5轮训练结束****
第5轮训练后,整体验证集上的Loss:165.31857767235488
第5轮训练后,整体验证集上的Accuracy:0.5681081081081081
————第6轮训练开始————
训练时间为：12.461488962173462, 总Loss:65.4671423882246
****第6轮训练结束****
第6轮训练后,整体验证集上的Loss:181.66920434311032
第6轮训练后,整体验证集上的Accuracy:0.5616216216216217
————第7轮训练开始————
训练时间为：12.4467